# Renormalization group

Given the quantum Ising Hamiltonian in transverse field on a one-dimensional lattice with nearest neighbor interaction:
$$
\hat{H} = \lambda \sum_{i=1}^{N} \sigma_z^{(i)} + \sum_{i=1}^{N-1} \sigma_x^{(i)} \sigma_x^{(i+1)}
$$

where $ \sigma_x $ and $ \sigma_z $ are the Pauli matrices and $ \lambda $ is the transverse field.

1. Compute the ground state energy as a function of the transverse field $ \lambda $ by means of the real-space RG algorithm.
2. *Optional:* Compute the ground state energy as a function of $ \lambda $ by means of the INFINITE DMRG algorithm. <br>
Compare the results between them and with the mean field solution.

In [21]:
# ===========================================================================================================
# IMPORT ZONE
# ===========================================================================================================

import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp

# ===========================================================================================================
# ISING MODEL
# ===========================================================================================================

def pauli_matrices():
  """
  pauli_matrices:
    Builds the Pauli matrices as sparse matrices.

  Returns
  -------
  s_x, s_y, s_z: tuple of sp.csr_matrix
    Pauli matrices for a 2x2 system in sparse format.
  """
  s_x = sp.csr_matrix([[0, 1], [1, 0]], dtype=complex)
  s_y = sp.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
  s_z = sp.csr_matrix([[1, 0], [0, -1]], dtype=complex)
  return s_x, s_y, s_z

# ===========================================================================================================

def ising_hamiltonian(N, l):
  """
  ising_hamiltonian:
    Builds the Ising model Hamiltonian using sparse matrices.

  Parameters
  ----------
  N : int
    Number of spins.
  l : float
    Interaction strength.

  Returns
  -------
  H : sp.csr_matrix
    Sparse Ising Hamiltonian.
  """
  dim = 2 ** N
  H_nonint = sp.csr_matrix((dim, dim), dtype=complex)
  H_int = sp.csr_matrix((dim, dim), dtype=complex)
  
  s_x, _, s_z = pauli_matrices()
  
  for i in range(N):
    zterm = sp.kron(sp.identity(2**i, format='csr'), sp.kron(s_z, sp.identity(2**(N - i - 1), format='csr')))
    H_nonint += zterm
    
  for i in range(N - 1):
    xterm = sp.kron(sp.identity(2**i, format='csr'), sp.kron(s_x, sp.kron(s_x, sp.identity(2**(N - i - 2), format='csr'))))
    H_int += xterm
  
  H = H_int + l * H_nonint
  return H

# ===========================================================================================================

def projector(H, N):
  _, eigvec = sp.linalg.eigsh(H, k=2**N, which='SA')  # Compute the smallest `k` eigenvalues
  eigvec = eigvec.T
    
  proj = sp.csr_matrix((H.shape[0], H.shape[0]))
  
  for i in range(2**N):
    vec = sp.csr_matrix(eigvec[i]).T  # Convert the eigenvector to a sparse column
    proj += vec @ vec.T  # Sparse outer product

  return proj

# ===========================================================================================================

def initialize_A_B(N):
  s_x, _, _ = pauli_matrices()
  
  A_0 = sp.kron(sp.identity(2**(N - 1), format='csr'), s_x)
  B_0 = sp.kron(s_x, sp.identity(2**(N - 1), format='csr'))
  
  return A_0, B_0

# ===========================================================================================================

def compute_H_2N(N, H, A, B):  
  H_2N = sp.kron(H, sp.identity(2**(N), format='csr')) + sp.kron(sp.identity(2**(N), format='csr'), H) + sp.kron(A, B)
  return H_2N

# ===========================================================================================================

def update_operators(N, H_2Nn, An, Bn, Pn):
  Pn_dagger = Pn.conj().T
  I_N = sp.identity(2**N, format='csr')

  # Compute H_Nnn, Ann, Bnn, Pnn
  H_Nnn = 1 / 2 * Pn_dagger @ H_2Nn @ Pn
  Ann = 1 / np.sqrt(2) * Pn_dagger @ sp.kron(I_N, An) @ Pn
  Bnn = 1 / np.sqrt(2) * Pn_dagger @ sp.kron(Bn, I_N) @ Pn
  
  Pnn = projector(H_Nnn, N)
  
  return H_Nnn, Ann, Bnn, Pnn

# ===========================================================================================================
  
def update_hamiltonian(N, l, threshold, max_iter=100):
  prev_energy_density = None
  H = ising_hamiltonian(N, l)
  A, B = initialize_A_B(N)
  
  for iteration in range(1, max_iter + 1):
    # Build Hamiltonian of size 2N and projector
    H_2N = compute_H_2N(N, H, A, B)
    P = projector(H_2N, N)

    # Compute the current energy density
    eigenvalues, _ = sp.linalg.eigsh(H, k=1, which='SA')
    current_energy_density = eigenvalues[0] / N

    # Check for convergence if this is not the first iteration
    if prev_energy_density is not None:
      delta = abs(current_energy_density - prev_energy_density)
      
      if delta > threshold:
        N = H_2N.shape[0]
        H, A, B, P = update_operators(N, H_2N, A, B, P)
      else:
        print(f"Convergence achieved at iteration {iteration}: ε = {current_energy_density}")
        return current_energy_density

    # Update previous energy density for next iteration
    prev_energy_density = current_energy_density

    if iteration % 10 == 0:
      print(f"Iteration {iteration}: ε = {current_energy_density}")

  # If convergence is not reached within max_iter
  raise ValueError(f"Convergence not achieved within {max_iter} iterations.")

In [24]:
# Parameters
N = 30
l = 0.5
threshold = 1e-15
max_iter = 50

energy_density = update_hamiltonian(N, l, threshold, max_iter)


: 